# NIRCam Coronagraphy plotting notebook

Author: Jonathan Aguilar (jaguilar@stsci.edu)

This notebook filters the enormous number of NIRCam aperture options to just the ones relevant for cornagraphy. For a more general introduction to WHIPPOT, see [WHIPPOT_tutorial](WHIPPOT_tutorial).

The user provides the RA and Dec coordinates for the target acquisition (ACQ) star and science (SCI) star, and a telescope roll angle. These tools will then compute a) the "Special Requirment Offset" X and Y values needed to place the SCI star at the reference position, and b) the final coordinates of the SCI and ACQ stars. Coordinates are given in IDL (https://jwst-docs.stsci.edu/jwst-observatory-hardware/jwst-target-observability-and-observatory-coordinate-system/jwst-instrument-ideal-coordinate-systems#gsc.tab=0).

The user can also provide a list of extra targets in the field in the format specified below, and their IDL coordinates will also be computed. As such, this notebook is useful not just for planning target acquisition, but also for predicting to high precision the final positions of astronomical bodies in the field of view.

## Setup ##

- With installing the module:
  1. Navigate in a terminal to the directory where you have downloaded the module. This should contain the `setup.py` file.
  2. Activate your desired python environment.
  3. Run `pip install .`
  4. Copy this notebook to your working directory, and run your jupyter kernel in the appropriate environment.
- Without installing the module:
  1. Copy this notebook and `utils.py` to your working directory.
  2. In the cell below, change `from all_jwst_offset_ta import utils` to `import utils`
  3. Open the notebook as you normally would. The following python modules must be installed in the working enviroment:
     -  `ipywidgets`
     - `numpy`
     - `matplotlib`
     - `astropy`
     - `pysiaf`

In [ ]:
# for development purposes
%load_ext autoreload
%autoreload 2

In [ ]:
# for your system, choose an appropriate plotting backend to make pop-up window plots
# (currently, inline plots show up in the log but not directly in the notebook).
# %matplotlib throws an error if you run it first, so do inline first.
%matplotlib inline
%matplotlib osx

In [ ]:
from whippot import whippot_tools
from whippot.modes import nircam_coron_tools
# if the library is not installed replace the line above with the line below 
# when `whippot_tools.py` is in the same folder as the notebook
# import whippot_tools

## An example of a real field

The source coordinates below are taken from a real commissioning activity, where we needed to check that the traces wouldn't overlap.

In [ ]:
# some 
# HIP 99770: 20 14 32.0323540824 +36 48 22.700909124
hip99770 = whippot_tools.SkyCoord("20 14 32.0323540824 +36 48 22.700909124", frame='icrs', unit=("hourangle","degree"))
# let's say the companion has an offset of dRA = 0.5 arcsec, dDec = 0.5 arcsec
from astropy import units
companion = hip99770.spherical_offsets_by(1*units.arcsec, 1*units.arcsec)
sources = {
    'SCI': hip99770,
    'b': companion,
}

In [ ]:
# format the sources for easy copy-pasting
for s, c in sources.items():
    print(f"{s}: ({c.ra.deg}, {c.dec.deg})")

In [ ]:
from astropy import units
initial_values={
    'instr': 'nircam',
    'sci_aper': 'nrca5_mask335r', 
    'pa': 290.,
    'sci_ra': sources['SCI'].ra.deg, 'sci_dec': sources['SCI'].dec.deg,
    'sci_ra': sources['SCI'].ra.deg, 'sci_dec': sources['SCI'].dec.deg,
    'exclude_roi': True,
}
# add a multi-line string of the other stars, copied from the cell above
initial_values['other_stars'] = "\n".join(f"{k}: ({v.ra.deg}, {v.dec.deg})" for k, v in sources.items() if k not in ['SCI'])

## Use the NIRCam Coronagraphy Tools module

We have subclassed the ComputePositions() object from `whippot_tools` (see the `WHIPPOT_tutorial.ipynb` notebook) and modified it to better suit NIRCam users.

In [ ]:
cp1 = nircam_coron_tools.ComputePositions(initial_values=initial_values)
cp1.ui

In [ ]:
a = 'asdfafd'
a.endswith('f')